In [1]:
# Real AWS Big Data Analytics Implementation
import boto3
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

print("🔧 REAL AWS IMPLEMENTATION")
print("⚠️  WARNING: This will incur AWS costs!")
print("💰 Estimated cost: $5-15 for full run")
print("🛑 Make sure to terminate resources after use!")

# AWS Credentials Setup (Choose ONE method)
print("\n🔑 Setting up AWS credentials...")


# Add Keys
# os.environ['AWS_ACCESS_KEY_ID'] = '-actual-access-key'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'actual-secret-key'
# os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

# Method 2: Check if AWS CLI is configured
def check_aws_credentials():
    try:
        session = boto3.Session()
        credentials = session.get_credentials()
        if credentials and credentials.access_key:
            print("✅ AWS credentials found")
            return True
        else:
            print("❌ No AWS credentials found")
            return False
    except Exception as e:
        print(f"❌ AWS credential check failed: {e}")
        return False

# Check credentials
aws_configured = check_aws_credentials()

if not aws_configured:
    print("\n🚨 AWS CREDENTIALS REQUIRED!")
    print("Please set up credentials using one of these methods:")
    print("1. Run: aws configure")
    print("2. Set environment variables (uncomment lines above)")
    print("3. Use IAM roles (if running on EC2)")
    print("\n⏸️  Stopping execution - configure credentials first")
    # Uncomment the next line to stop execution
    # raise Exception("AWS credentials required")
else:
    print("✅ Ready to use real AWS services!")

🔧 REAL AWS IMPLEMENTATION
⚠️  WARNING: This will incur AWS costs!
💰 Estimated cost: $5-15 for full run
🛑 Make sure to terminate resources after use!

🔑 Setting up AWS credentials...
❌ No AWS credentials found

🚨 AWS CREDENTIALS REQUIRED!
Please set up credentials using one of these methods:
1. Run: aws configure
2. Set environment variables (uncomment lines above)
3. Use IAM roles (if running on EC2)

⏸️  Stopping execution - configure credentials first


In [ ]:
# Real AWS Configuration
AWS_CONFIG = {
    'region': 'us-east-1',
    'bucket_name': f'misinformation-detection-{datetime.now().strftime("%Y%m%d-%H%M%S")}',  # Unique bucket name
    'athena_database': 'misinformation_db',
    'athena_results_location': '',  # Will be set after bucket creation
    'emr_log_uri': '',  # Will be set after bucket creation
    'sagemaker_role': '',  # Will need to be configured
}

# Initialize AWS clients with error handling
def initialize_aws_clients():
    try:
        clients = {
            's3': boto3.client('s3', region_name=AWS_CONFIG['region']),
            'athena': boto3.client('athena', region_name=AWS_CONFIG['region']),
            'emr': boto3.client('emr', region_name=AWS_CONFIG['region']),
            'sagemaker': boto3.client('sagemaker', region_name=AWS_CONFIG['region']),
            'iam': boto3.client('iam', region_name=AWS_CONFIG['region'])
        }
        
        # Test S3 connection
        clients['s3'].list_buckets()
        print("✅ All AWS clients initialized successfully")
        return clients, True
        
    except Exception as e:
        print(f"❌ Failed to initialize AWS clients: {e}")
        return None, False

# Initialize clients
aws_clients, aws_ready = initialize_aws_clients()

if aws_ready:
    print(f"✅ AWS region: {AWS_CONFIG['region']}")
    print(f"✅ Bucket name: {AWS_CONFIG['bucket_name']}")
else:
    print("❌ AWS not ready - check credentials and permissions")

In [ ]:
# Real S3 Data Lake Implementation
def create_s3_data_lake():
    """Create real S3 bucket and upload data"""
    if not aws_ready:
        print("❌ AWS not ready - skipping S3 setup")
        return False
    
    s3_client = aws_clients['s3']
    bucket_name = AWS_CONFIG['bucket_name']
    
    try:
        # Create S3 bucket
        print(f"🗄️ Creating S3 bucket: {bucket_name}")
        
        if AWS_CONFIG['region'] == 'us-east-1':
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': AWS_CONFIG['region']}
            )
        
        print(f"✅ S3 bucket created: {bucket_name}")
        
        # Configure bucket for analytics
        s3_client.put_bucket_versioning(
            Bucket=bucket_name,
            VersioningConfiguration={'Status': 'Enabled'}
        )
        
        # Set up folder structure
        folders = [
            'raw_data/',
            'processed_data/',
            'model_artifacts/',
            'athena_results/',
            'emr_logs/',
            'sagemaker_output/'
        ]
        
        for folder in folders:
            s3_client.put_object(Bucket=bucket_name, Key=folder)
        
        print("✅ S3 folder structure created")
        
        # Update configuration with S3 paths
        AWS_CONFIG['athena_results_location'] = f's3://{bucket_name}/athena_results/'
        AWS_CONFIG['emr_log_uri'] = f's3://{bucket_name}/emr_logs/'
        
        return True
        
    except Exception as e:
        print(f"❌ Failed to create S3 bucket: {e}")
        return False

def upload_datasets_to_s3():
    """Upload real datasets to S3"""
    if not aws_ready:
        return False
    
    s3_client = aws_clients['s3']
    bucket_name = AWS_CONFIG['bucket_name']
    
    try:
        # Load local dataset
        df = pd.read_csv('../data/processed/misinformation_dataset.csv')
        print(f"📊 Uploading dataset: {len(df)} records")
        
        # Upload CSV format
        csv_buffer = df.to_csv(index=False)
        s3_client.put_object(
            Bucket=bucket_name,
            Key='raw_data/misinformation_dataset.csv',
            Body=csv_buffer
        )
        
        # Upload Parquet format (better for analytics)
        parquet_buffer = df.to_parquet(index=False)
        s3_client.put_object(
            Bucket=bucket_name,
            Key='processed_data/misinformation_dataset.parquet',
            Body=parquet_buffer
        )
        
        # Upload model results if available
        try:
            with open('../results/model_results.json', 'r') as f:
                model_results = f.read()
            
            s3_client.put_object(
                Bucket=bucket_name,
                Key='model_artifacts/model_results.json',
                Body=model_results
            )
            print("✅ Model results uploaded")
        except FileNotFoundError:
            print("⚠️ Model results not found - run notebook 02 first")
        
        print("✅ All datasets uploaded to S3")
        
        # Verify uploads
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        print(f"📁 S3 objects created: {response.get('KeyCount', 0)}")
        
        return True
        
    except Exception as e:
        print(f"❌ Failed to upload datasets: {e}")
        return False

# Create S3 data lake
print("🗄️ SETTING UP S3 DATA LAKE")
print("-" * 40)
s3_created = create_s3_data_lake()

if s3_created:
    data_uploaded = upload_datasets_to_s3()
    if data_uploaded:
        print("✅ S3 data lake setup complete")
    else:
        print("⚠️ S3 created but data upload failed")
else:
    print("❌ S3 setup failed")

In [ ]:
# Real AWS Athena Implementation
def setup_athena_database():
    """Create real Athena database and tables"""
    if not aws_ready:
        return False
    
    athena_client = aws_clients['athena']
    
    try:
        # Create database
        create_db_query = f"""
        CREATE DATABASE IF NOT EXISTS {AWS_CONFIG['athena_database']}
        COMMENT 'Misinformation detection analytics database'
        """
        
        print("🔍 Creating Athena database...")
        response = athena_client.start_query_execution(
            QueryString=create_db_query,
            QueryExecutionContext={'Database': 'default'},
            ResultConfiguration={
                'OutputLocation': AWS_CONFIG['athena_results_location']
            }
        )
        
        # Wait for query completion
        query_id = response['QueryExecutionId']
        wait_for_query_completion(athena_client, query_id)
        
        # Create external table
        create_table_query = f"""
        CREATE EXTERNAL TABLE IF NOT EXISTS {AWS_CONFIG['athena_database']}.misinformation_data (
            text string,
            label bigint,
            source string
        )
        STORED AS PARQUET
        LOCATION 's3://{AWS_CONFIG["bucket_name"]}/processed_data/'
        """
        
        print("📊 Creating Athena table...")
        response = athena_client.start_query_execution(
            QueryString=create_table_query,
            QueryExecutionContext={'Database': AWS_CONFIG['athena_database']},
            ResultConfiguration={
                'OutputLocation': AWS_CONFIG['athena_results_location']
            }
        )
        
        query_id = response['QueryExecutionId']
        wait_for_query_completion(athena_client, query_id)
        
        print("✅ Athena database and table created")
        return True
        
    except Exception as e:
        print(f"❌ Failed to setup Athena: {e}")
        return False

def wait_for_query_completion(athena_client, query_id, max_wait=300):
    """Wait for Athena query to complete"""
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        response = athena_client.get_query_execution(QueryExecutionId=query_id)
        status = response['QueryExecution']['Status']['State']
        
        if status in ['SUCCEEDED']:
            return True
        elif status in ['FAILED', 'CANCELLED']:
            print(f"❌ Query {status}: {response['QueryExecution']['Status'].get('StateChangeReason', '')}")
            return False
        
        time.sleep(2)
    
    print(f"⏰ Query timeout after {max_wait} seconds")
    return False

def run_athena_analytics():
    """Run real Athena analytics queries"""
    if not aws_ready:
        return None
    
    athena_client = aws_clients['athena']
    
    queries = {
        'total_records': f"SELECT COUNT(*) as total_count FROM {AWS_CONFIG['athena_database']}.misinformation_data",
        'label_distribution': f"SELECT label, COUNT(*) as count FROM {AWS_CONFIG['athena_database']}.misinformation_data GROUP BY label",
        'avg_text_length': f"SELECT label, AVG(LENGTH(text)) as avg_length FROM {AWS_CONFIG['athena_database']}.misinformation_data GROUP BY label",
        'source_analysis': f"SELECT source, COUNT(*) as count FROM {AWS_CONFIG['athena_database']}.misinformation_data GROUP BY source"
    }
    
    results = {}
    total_cost = 0
    
    print("🔍 Running Athena analytics queries...")
    
    for query_name, query_sql in queries.items():
        try:
            print(f"  📊 Executing: {query_name}")
            
            response = athena_client.start_query_execution(
                QueryString=query_sql,
                QueryExecutionContext={'Database': AWS_CONFIG['athena_database']},
                ResultConfiguration={
                    'OutputLocation': AWS_CONFIG['athena_results_location']
                }
            )
            
            query_id = response['QueryExecutionId']
            
            if wait_for_query_completion(athena_client, query_id):
                # Get query results
                result_response = athena_client.get_query_results(QueryExecutionId=query_id)
                
                # Get query statistics for cost calculation
                stats_response = athena_client.get_query_execution(QueryExecutionId=query_id)
                data_scanned = stats_response['QueryExecution']['Statistics'].get('DataScannedInBytes', 0)
                cost = (data_scanned / (1024**4)) * 5  # $5 per TB
                total_cost += cost
                
                # Process results
                rows = result_response['ResultSet']['Rows']
                if len(rows) > 1:  # Skip header row
                    data_rows = []
                    for row in rows[1:]:  # Skip header
                        data_rows.append([col.get('VarCharValue', '') for col in row['Data']])
                    results[query_name] = {
                        'data': data_rows,
                        'data_scanned_bytes': data_scanned,
                        'cost_usd': cost
                    }
                
                print(f"    ✅ Completed - Cost: ${cost:.4f}")
            else:
                print(f"    ❌ Failed")
                
        except Exception as e:
            print(f"    ❌ Error: {e}")
    
    print(f"\n💰 Total Athena cost: ${total_cost:.4f}")
    return results

# Setup and run Athena
print("🔍 SETTING UP ATHENA ANALYTICS")
print("-" * 40)

if s3_created:
    athena_setup = setup_athena_database()
    
    if athena_setup:
        athena_results = run_athena_analytics()
        if athena_results:
            print("✅ Athena analytics completed")
            print(f"📊 Queries executed: {len(athena_results)}")
        else:
            print("⚠️ Athena setup complete but queries failed")
    else:
        print("❌ Athena setup failed")
else:
    print("⚠️ Skipping Athena - S3 not ready")

In [ ]:
# Real AWS EMR (Elastic MapReduce) with Spark Implementation
def create_emr_cluster():
    """Create real EMR cluster for Spark processing"""
    if not aws_ready:
        return None
    
    emr_client = aws_clients['emr']
    
    try:
        print("⚡ Creating EMR cluster for Spark processing...")
        print("💰 Estimated cost: $0.50-1.00 per hour")
        
        # EMR cluster configuration
        cluster_config = {
            'Name': f'misinformation-detection-{datetime.now().strftime("%Y%m%d-%H%M")}',
            'ReleaseLabel': 'emr-6.15.0',
            'Instances': {
                'InstanceGroups': [
                    {
                        'Name': 'Master nodes',
                        'Market': 'ON_DEMAND',
                        'InstanceRole': 'MASTER',
                        'InstanceType': 'm5.xlarge',
                        'InstanceCount': 1,
                    },
                    {
                        'Name': 'Worker nodes',
                        'Market': 'SPOT',  # Use spot instances for cost savings
                        'InstanceRole': 'CORE',
                        'InstanceType': 'm5.large',
                        'InstanceCount': 2,
                        'BidPrice': '0.05'  # Max bid for spot instances
                    }
                ],
                'KeepJobFlowAliveWhenNoSteps': True,
                'TerminationProtected': False  # Allow termination
            },
            'Applications': [
                {'Name': 'Spark'},
                {'Name': 'Hadoop'},
                {'Name': 'Livy'}  # For notebook integration
            ],
            'LogUri': AWS_CONFIG['emr_log_uri'],
            'ServiceRole': 'EMR_DefaultRole',
            'JobFlowRole': 'EMR_EC2_DefaultRole',
            'VisibleToAllUsers': True
        }
        
        response = emr_client.run_job_flow(**cluster_config)
        cluster_id = response['JobFlowId']
        
        print(f"✅ EMR cluster created: {cluster_id}")
        print("⏳ Waiting for cluster to start (5-10 minutes)...")
        
        # Wait for cluster to be ready
        waiter = emr_client.get_waiter('cluster_running')
        waiter.wait(
            ClusterId=cluster_id,
            WaiterConfig={'Delay': 30, 'MaxAttempts': 20}
        )
        
        print("✅ EMR cluster is running and ready!")
        return cluster_id
        
    except Exception as e:
        print(f"❌ Failed to create EMR cluster: {e}")
        return None

def submit_spark_job(cluster_id):
    """Submit Spark job to EMR cluster"""
    if not aws_ready or not cluster_id:
        return None
    
    emr_client = aws_clients['emr']
    
    try:
        print("📊 Submitting Spark job for misinformation processing...")
        
        # Spark application code (stored in S3)
        spark_script = f"""
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import time

# Initialize Spark
spark = SparkSession.builder.appName("MisinformationDetection").getOrCreate()

# Read data from S3
df = spark.read.option("header", "true").csv("s3://{AWS_CONFIG['bucket_name']}/raw_data/misinformation_dataset.csv")
df = df.withColumn("label", df.label.cast("double"))

print(f"Loaded dataset with {{df.count()}} records")

# Create ML pipeline
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=1000)
idf = IDF(inputCol="rawFeatures", outputCol="features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20)

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, rf])

# Split data
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Train model
start_time = time.time()
model = pipeline.fit(train_data)
training_time = time.time() - start_time

# Make predictions
predictions = model.transform(test_data)

# Evaluate
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Training completed in {{training_time:.2f}} seconds")
print(f"Model accuracy: {{accuracy:.4f}}")

# Save results to S3
results_df = spark.createDataFrame([{{
    "training_time": training_time,
    "accuracy": accuracy,
    "records_processed": df.count(),
    "throughput": df.count() / training_time
}}])

results_df.write.mode("overwrite").json("s3://{AWS_CONFIG['bucket_name']}/emr_results/")

spark.stop()
"""
        
        # Upload Spark script to S3
        s3_client = aws_clients['s3']
        s3_client.put_object(
            Bucket=AWS_CONFIG['bucket_name'],
            Key='scripts/spark_misinformation_job.py',
            Body=spark_script
        )
        
        # Submit step to EMR
        step_config = {
            'Name': 'Misinformation Detection Spark Job',
            'ActionOnFailure': 'CONTINUE',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': [
                    'spark-submit',
                    '--deploy-mode', 'cluster',
                    f's3://{AWS_CONFIG["bucket_name"]}/scripts/spark_misinformation_job.py'
                ]
            }
        }
        
        response = emr_client.add_job_flow_steps(
            JobFlowId=cluster_id,
            Steps=[step_config]
        )
        
        step_id = response['StepIds'][0]
        print(f"✅ Spark job submitted: {step_id}")
        
        # Wait for step completion
        print("⏳ Waiting for Spark job to complete...")
        waiter = emr_client.get_waiter('step_complete')
        waiter.wait(
            ClusterId=cluster_id,
            StepId=step_id,
            WaiterConfig={'Delay': 30, 'MaxAttempts': 20}
        )
        
        print("✅ Spark job completed!")
        return step_id
        
    except Exception as e:
        print(f"❌ Failed to submit Spark job: {e}")
        return None

def cleanup_emr_cluster(cluster_id):
    """Terminate EMR cluster to avoid costs"""
    if not aws_ready or not cluster_id:
        return
    
    try:
        print(f"🧹 Terminating EMR cluster: {cluster_id}")
        emr_client = aws_clients['emr']
        emr_client.terminate_job_flows(JobFlowIds=[cluster_id])
        print("✅ EMR cluster termination initiated")
        print("💰 Billing will stop once termination completes")
    except Exception as e:
        print(f"❌ Failed to terminate cluster: {e}")

# Run EMR Spark processing
print("⚡ SETTING UP EMR SPARK PROCESSING")
print("-" * 40)

if s3_created:
    cluster_id = create_emr_cluster()
    
    if cluster_id:
        step_id = submit_spark_job(cluster_id)
        
        if step_id:
            print("✅ EMR Spark processing completed")
            
            # Cleanup (IMPORTANT for cost control)
            cleanup_choice = input("🛑 Terminate EMR cluster now? (y/n): ")
            if cleanup_choice.lower() == 'y':
                cleanup_emr_cluster(cluster_id)
            else:
                print(f"⚠️ Remember to terminate cluster manually: {cluster_id}")
                print("💰 Cluster costs ~$0.50/hour while running!")
        else:
            print("❌ Spark job failed")
            cleanup_emr_cluster(cluster_id)
    else:
        print("❌ EMR cluster creation failed")
else:
    print("⚠️ Skipping EMR - S3 not ready")

In [ ]:
# Real AWS SageMaker Implementation
def setup_sagemaker_role():
    """Create or get SageMaker execution role"""
    if not aws_ready:
        return None
    
    iam_client = aws_clients['iam']
    
    try:
        # Check if role exists
        role_name = 'MisinformationDetectionSageMakerRole'
        
        try:
            response = iam_client.get_role(RoleName=role_name)
            role_arn = response['Role']['Arn']
            print(f"✅ Using existing SageMaker role: {role_arn}")
            return role_arn
        except iam_client.exceptions.NoSuchEntityException:
            pass
        
        # Create role if it doesn't exist
        print("🔧 Creating SageMaker execution role...")
        
        trust_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {"Service": "sagemaker.amazonaws.com"},
                    "Action": "sts:AssumeRole"
                }
            ]
        }
        
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description='SageMaker execution role for misinformation detection'
        )
        
        role_arn = response['Role']['Arn']
        
        # Attach necessary policies
        policies = [
            'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
            'arn:aws:iam::aws:policy/AmazonS3FullAccess'
        ]
        
        for policy_arn in policies:
            iam_client.attach_role_policy(
                RoleName=role_name,
                PolicyArn=policy_arn
            )
        
        print(f"✅ SageMaker role created: {role_arn}")
        time.sleep(10)  # Wait for role propagation
        
        return role_arn
        
    except Exception as e:
        print(f"❌ Failed to setup SageMaker role: {e}")
        return None

def create_sagemaker_training_job():
    """Create real SageMaker training job"""
    if not aws_ready:
        return None
    
    # Setup role
    role_arn = setup_sagemaker_role()
    if not role_arn:
        return None
    
    sagemaker_client = aws_clients['sagemaker']
    
    try:
        print("🤖 Creating SageMaker training job...")
        print("💰 Estimated cost: $2-5 for training")
        
        # Training script (simplified for demo)
        training_script = '''
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib
import boto3
import os

def train():
    # Download data from S3
    s3 = boto3.client('s3')
    bucket = os.environ['BUCKET_NAME']
    
    s3.download_file(bucket, 'raw_data/misinformation_dataset.csv', '/tmp/data.csv')
    
    # Load and process data
    df = pd.read_csv('/tmp/data.csv')
    
    # Feature extraction
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    X = vectorizer.fit_transform(df['text'])
    y = df['label']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Model accuracy: {accuracy:.4f}")
    
    # Save model
    joblib.dump(model, '/opt/ml/model/model.pkl')
    joblib.dump(vectorizer, '/opt/ml/model/vectorizer.pkl')
    
    # Save metrics
    with open('/opt/ml/model/metrics.json', 'w') as f:
        json.dump({'accuracy': accuracy}, f)

if __name__ == '__main__':
    train()
'''
        
        # Upload training script to S3
        s3_client = aws_clients['s3']
        s3_client.put_object(
            Bucket=AWS_CONFIG['bucket_name'],
            Key='sagemaker_code/train.py',
            Body=training_script
        )
        
        # Create training job
        job_name = f"misinformation-training-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
        
        training_config = {
            'TrainingJobName': job_name,
            'RoleArn': role_arn,
            'AlgorithmSpecification': {
                'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
                'TrainingInputMode': 'File'
            },
            'InputDataConfig': [
                {
                    'ChannelName': 'training',
                    'DataSource': {
                        'S3DataSource': {
                            'S3DataType': 'S3Prefix',
                            'S3Uri': f"s3://{AWS_CONFIG['bucket_name']}/raw_data/",
                            'S3DataDistributionType': 'FullyReplicated'
                        }
                    }
                }
            ],
            'OutputDataConfig': {
                'S3OutputPath': f"s3://{AWS_CONFIG['bucket_name']}/sagemaker_output/"
            },
            'ResourceConfig': {
                'InstanceType': 'ml.m5.large',  # Cost-effective instance
                'InstanceCount': 1,
                'VolumeSizeInGB': 10
            },
            'StoppingCondition': {
                'MaxRuntimeInSeconds': 3600  # 1 hour max
            },
            'Environment': {
                'BUCKET_NAME': AWS_CONFIG['bucket_name']
            },
            'HyperParameters': {
                'n_estimators': '100',
                'random_state': '42'
            }
        }
        
        response = sagemaker_client.create_training_job(**training_config)
        
        print(f"✅ SageMaker training job created: {job_name}")
        print("⏳ Waiting for training to complete...")
        
        # Wait for completion
        waiter = sagemaker_client.get_waiter('training_job_completed_or_stopped')
        waiter.wait(
            TrainingJobName=job_name,
            WaiterConfig={'Delay': 30, 'MaxAttempts': 60}
        )
        
        # Get training job status
        response = sagemaker_client.describe_training_job(TrainingJobName=job_name)
        status = response['TrainingJobStatus']
        
        if status == 'Completed':
            print("✅ SageMaker training completed successfully!")
            
            # Get training metrics
            training_time = response['TrainingTimeInSeconds']
            billable_time = response['BillableTimeInSeconds']
            cost = (billable_time / 3600) * 0.134  # ml.m5.large cost
            
            print(f"📊 Training time: {training_time} seconds")
            print(f"💰 Estimated cost: ${cost:.2f}")
            
            return job_name
        else:
            print(f"❌ SageMaker training failed: {status}")
            return None
            
    except Exception as e:
        print(f"❌ Failed to create SageMaker training job: {e}")
        return None

# Run SageMaker training
print("🤖 SETTING UP SAGEMAKER TRAINING")
print("-" * 40)

if s3_created:
    sagemaker_job = create_sagemaker_training_job()
    
    if sagemaker_job:
        print("✅ SageMaker training completed")
    else:
        print("❌ SageMaker training failed")
else:
    print("⚠️ Skipping SageMaker - S3 not ready")

In [ ]:
# Cost Monitoring and Resource Cleanup
def calculate_actual_costs():
    """Calculate actual AWS costs incurred"""
    if not aws_ready:
        return
    
    print("💰 COST ANALYSIS")
    print("-" * 30)
    
    costs = {
        'S3 Storage': 0.023 * 0.001,  # ~1MB of data
        'Athena Queries': 0.05,  # Estimated based on data scanned
        'EMR Cluster': 0.50,  # 1 hour of cluster time
        'SageMaker Training': 2.00,  # ml.m5.large for ~15 minutes
        'Data Transfer': 0.10  # Various transfers
    }
    
    total_cost = sum(costs.values())
    
    print("Estimated costs for this run:")
    for service, cost in costs.items():
        print(f"  {service}: ${cost:.2f}")
    
    print(f"\n💵 Total estimated cost: ${total_cost:.2f}")
    print("\n💡 Cost optimization tips:")
    print("  • Use spot instances for EMR (50-70% savings)")
    print("  • Terminate resources immediately after use")
    print("  • Use S3 lifecycle policies for data management")
    print("  • Monitor costs with AWS Budgets")

def cleanup_all_resources():
    """Clean up all AWS resources to stop billing"""
    if not aws_ready:
        return
    
    print("🧹 CLEANING UP AWS RESOURCES")
    print("-" * 35)
    
    s3_client = aws_clients['s3']
    bucket_name = AWS_CONFIG['bucket_name']
    
    try:
        # List active EMR clusters
        emr_client = aws_clients['emr']
        clusters = emr_client.list_clusters(
            ClusterStates=['STARTING', 'BOOTSTRAPPING', 'RUNNING', 'WAITING']
        )
        
        for cluster in clusters['Clusters']:
            if 'misinformation-detection' in cluster['Name']:
                print(f"🛑 Terminating EMR cluster: {cluster['Id']}")
                emr_client.terminate_job_flows(JobFlowIds=[cluster['Id']])
        
        # Optionally delete S3 bucket (ask user)
        cleanup_s3 = input(f"🗑️ Delete S3 bucket '{bucket_name}' and all data? (y/n): ")
        
        if cleanup_s3.lower() == 'y':
            # Delete all objects first
            print("🗑️ Deleting S3 objects...")
            response = s3_client.list_objects_v2(Bucket=bucket_name)
            
            if 'Contents' in response:
                objects = [{'Key': obj['Key']} for obj in response['Contents']]
                s3_client.delete_objects(
                    Bucket=bucket_name,
                    Delete={'Objects': objects}
                )
            
            # Delete bucket
            s3_client.delete_bucket(Bucket=bucket_name)
            print(f"✅ S3 bucket '{bucket_name}' deleted")
        else:
            print(f"⚠️ S3 bucket '{bucket_name}' preserved")
            print("   Remember: S3 storage costs ~$0.023/GB/month")
        
        print("✅ Cleanup completed!")
        
    except Exception as e:
        print(f"❌ Cleanup error: {e}")

# Run cost analysis
calculate_actual_costs()

# Ask user about cleanup
print("\n" + "="*50)
cleanup_now = input("🧹 Clean up AWS resources now? (y/n): ")

if cleanup_now.lower() == 'y':
    cleanup_all_resources()
else:
    print("⚠️ Remember to clean up resources manually to avoid ongoing charges!")
    print(f"   - Terminate any running EMR clusters")
    print(f"   - Delete S3 bucket: {AWS_CONFIG['bucket_name']}")
    print(f"   - Check SageMaker for running endpoints")

In [ ]:
# Real AWS Implementation Results Summary
print("🎉 REAL AWS IMPLEMENTATION COMPLETE!")
print("="*60)

print("\n✅ SERVICES SUCCESSFULLY USED:")
services_used = []
if s3_created:
    services_used.append("✅ S3 Data Lake")
if 'athena_results' in locals() and athena_results:
    services_used.append("✅ Athena Analytics")
if 'cluster_id' in locals() and cluster_id:
    services_used.append("✅ EMR Spark Processing")
if 'sagemaker_job' in locals() and sagemaker_job:
    services_used.append("✅ SageMaker Training")

for service in services_used:
    print(f"  {service}")

print(f"\n📊 REAL AWS RESULTS:")
print(f"  • S3 Bucket: {AWS_CONFIG['bucket_name']}")
if 'athena_results' in locals() and athena_results:
    print(f"  • Athena Queries: {len(athena_results)} executed")
if 'cluster_id' in locals():
    print(f"  • EMR Cluster: {cluster_id}")
if 'sagemaker_job' in locals():
    print(f"  • SageMaker Job: {sagemaker_job}")

print(f"\n💰 ESTIMATED TOTAL COST: $5-15")
print(f"⚠️  Make sure all resources are terminated!")

print(f"\n📁 COMPARISON WITH SIMULATION:")
print(f"  • Simulation (Notebook 03): $0 cost, academic-friendly")
print(f"  • Real AWS (This notebook): Real costs, production experience")
print(f"  • Both demonstrate AWS knowledge effectively")

print(f"\n🎓 FOR YOUR ASSESSMENT:")
print(f"  • Use simulation version for submission (cost-free)")
print(f"  • Mention real AWS implementation for extra credit")
print(f"  • Include cost analysis in your evaluation")

print("="*60)